In [ ]:
#!pip install tensorflow
#!pip install keras
#!pip install --upgrade tensorflow
#!pip install --upgrade tensorflow-gpu

In [ ]:
import tensorflow
from tensorflow import keras
import numpy as np
import pandas as pd
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.datasets import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense, Flatten,Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
import itertools
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from sklearn.utils import class_weight
import tensorflow_addons as tfa
import pathlib
from tensorflow.keras import regularizers
#model = tensorflow.keras.models.load_model("../input/models/inception_val69167.h5")
#os.mkdir("/kaggle/working/models")
os.chdir("/kaggle/input/knee-osteoarthritis-dataset-with-severity")

In [ ]:
base_dir = "/kaggle/input/knee-osteoarthritis-dataset-with-severity"
train_path = os.path.join(base_dir,'train')
valid_path = os.path.join(base_dir,'val')
test_path = os.path.join(base_dir, 'test')

In [ ]:
import random
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from collections import Counter
import cv2

ig, ax = plt.subplots(5,5, figsize=(18,18))

for class_id in range(5):
    folder = os.path.join(train_path,str(class_id))
    os.chdir(folder)
    samples = random.sample(os.listdir(folder), 5)
    
    for col in range(5):
        image = cv2.imread(samples[col])
        ax[class_id, col].imshow(image)
        ax[class_id, col].set_title("class_" + str(class_id))
        ax[class_id, col].set_axis_off()
    
plt.show()

In [ ]:
def random_erasing(img, sl=0.1, sh=0.2, rl=0.4, p=0.4):
    h = tensorflow.shape(img)[0]
    w = tensorflow.shape(img)[1]
    c = tensorflow.shape(img)[2]
    origin_area = tensorflow.cast(h*w, tensorflow.float32)

    e_size_l = tensorflow.cast(tensorflow.round(tensorflow.sqrt(origin_area * sl * rl)), tensorflow.int32)
    e_size_h = tensorflow.cast(tensorflow.round(tensorflow.sqrt(origin_area * sh / rl)), tensorflow.int32)

    e_height_h = tensorflow.minimum(e_size_h, h)
    e_width_h = tensorflow.minimum(e_size_h, w)

    erase_height = tensorflow.random.uniform(shape=[], minval=e_size_l, maxval=e_height_h, dtype=tensorflow.int32)
    erase_width = tensorflow.random.uniform(shape=[], minval=e_size_l, maxval=e_width_h, dtype=tensorflow.int32)

    erase_area = tensorflow.zeros(shape=[erase_height, erase_width, c])
    erase_area = tensorflow.cast(erase_area, tensorflow.uint8)

    pad_h = h - erase_height
    pad_top = tensorflow.random.uniform(shape=[], minval=0, maxval=pad_h, dtype=tensorflow.int32)
    pad_bottom = pad_h - pad_top

    pad_w = w - erase_width
    pad_left = tensorflow.random.uniform(shape=[], minval=0, maxval=pad_w, dtype=tensorflow.int32)
    pad_right = pad_w - pad_left

    erase_mask = tensorflow.pad([erase_area], [[0,0],[pad_top, pad_bottom], [pad_left, pad_right], [0,0]], constant_values=1)
    erase_mask = tensorflow.squeeze(erase_mask, axis=0)
    erased_img = tensorflow.multiply(tensorflow.cast(img,tensorflow.float32), tensorflow.cast(erase_mask, tensorflow.float32))

    return tensorflow.cond(tensorflow.random.uniform([], 0, 1) > p, lambda: tensorflow.cast(img, img.dtype), lambda:  tensorflow.cast(erased_img, img.dtype))

In [ ]:
# Use GPU
sess = tensorflow.compat.v1.Session(config=tensorflow.compat.v1.ConfigProto(log_device_placement=True))

In [ ]:
base = tensorflow.keras.applications.DenseNet201(
        include_top=False,
        input_tensor=None,
        input_shape=None)
base.summary()

In [ ]:
'''
lr = tensorflow.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=9,
    verbose=2,
    mode="max",
    min_delta=0.0001,
    min_lr=0.000001
)
'''

In [ ]:
    x = base.output
    x= keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.4)(x)
    predictions =Dense(units=5, activation = 'softmax', kernel_regularizer=regularizers.l1_l2(l1=0.02, l2=0.02))(x)
    model = Model(inputs=base.input, outputs=predictions)
    model.compile(Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

In [ ]:
batch=32
train_batches= ImageDataGenerator(horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.3,rotation_range=40,
    width_shift_range=0.25,
    height_shift_range=0.15,
    zoom_range=0.2,
    preprocessing_function = random_erasing).flow_from_directory(train_path, target_size=(224,224), classes=['0','1','2','3','4'], batch_size=batch)
valid_batches= ImageDataGenerator().flow_from_directory(valid_path, target_size=(224,224), classes=['0','1','2','3','4'], batch_size=batch)
test_batches= ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=['0','1','2','3','4'], batch_size=batch, shuffle= False)

In [ ]:
'''
 class_weights = class_weight.compute_class_weight(
            'balanced',
             np.unique(train_batches.classes), 
             train_batches.classes)

 train_weights = dict(enumerate(class_weights))
 train_weights
 '''

In [ ]:
checkpoint = ModelCheckpoint(filepath='/kaggle/working/Best_DenseNet201.h5',verbose=2, save_best_only=True, monitor = 'val_accuracy')

e = tensorflow.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=20, verbose=2, restore_best_weights=True)

callbacks = [checkpoint, e]

start = datetime.now()

In [ ]:
history = model.fit(train_batches, steps_per_epoch=5778//batch, validation_data=valid_batches, validation_steps=826//batch, epochs=120, verbose=2, callbacks = callbacks)

duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
#model.save('/kaggle/working/densenet.h5')
#model = keras.models.load_model('/kaggle/working/Best_dense2.h5')

In [ ]:
model.evaluate(test_batches)

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Confusion Matrix and Classification Report
Y_pred = model.predict_generator(test_batches, 1656 // 32+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_batches.classes, y_pred))
print('Classification Report')
target_names = ['0','1','2','3','4']
print(classification_report(test_batches.classes, y_pred, target_names=target_names))